In [1]:
from __future__ import print_function
import urllib2
import json
import datetime
import csv
import time

# from alchemyapi import AlchemyAPI
import json
import sys
import os

In [2]:
app_id = ""
app_secret = "" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

# enter the name or id of the Facebook page that you want to scrape
page_id_list = []
    
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print (e)
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return response.read()

In [3]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data

In [4]:
def processFacebookPageFeedStatus(status):
    
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    link_description='' if 'description' not in status.keys() else status['description'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link'].encode('utf-8')
    
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']


    return (status_id,status['page'], status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares,status['sentiment'],status['score'],status['category'])                    


In [5]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_status.csv' % page_id, 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id",'page' , "status_message", "link_name", "status_type", "status_link",
           "status_published", "num_likes", "num_comments", "num_shares","sentiment","sentiment_score","category"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print ("Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime))
        
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                status['sentiment'] = 0
                status['score'] = 0
                status['page'] = page_id
                status['category']=0
                tempx = processFacebookPageFeedStatus(status)
                w.writerow(tempx)
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print ("%s Statuses Processed: %s" % (num_processed, datetime.datetime.now()))
                    
                if status.get('created_time','')<"2015-05-01":
    
                    has_next_page = False
                    break
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
                print ("\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime))
            else:
                has_next_page = False
                
        
        print ("\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime))


In [ ]:
for values in page_id_list:
    scrapeFacebookPageFeedStatus(values, access_token)